<a href="https://colab.research.google.com/github/thomasAmorrow/EX-Tools/blob/main/EX_ROV_Dive_Track_Smoothing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dive track smoothing algorithm for ROV D2 Data**

A simple python script that calculates a smoothed moving average of position data from ROV *Deep Discoverer*


### Record
Original code
> *Thomas Morrow, 10 February 2022*

# Upload csv file
Import your dive track data when prompted below the code snippet, the top few lines of your imported table will appear on successful import

In [ ]:
import pandas as pd

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
track = pd.read_csv(fn) # track is the name of the dataframe we imported from csv

track.head() # show first lines of your track to make sure things imported

# Visualize current track
Plot the dive track in 3-D for a visualization check

In [ ]:
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt

# the names of your columns above should correspond to these arrays
xdata = track.LON_DD 
ydata = track.LAT_DD
zdata = track.DEPTH
ttime = track.UNIXTIME

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(xdata, ydata, zdata, 'gray')

# Smoothe the dive track using a windowed average 
Windowed average algorithm

In [ ]:
import math

windtime = 60 * 5 # 60 seconds times 5 minutes - feel free to adjust window time

xdatawin=xdata.rolling(windtime).mean()
ydatawin=ydata.rolling(windtime).mean()
zdatawin=zdata.rolling(windtime).mean()

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(xdata, ydata, zdata, 'gray')
ax.scatter3D(xdatawin, ydatawin, zdatawin, c=ttime, cmap='viridis', s=1)

# Generate new csv output dive track
Download prompt appears at end of execution with new filename_smoothed.csv

In [ ]:
trackout=track.copy() # make a copy of the original data

# replace with smoothed track
trackout['LON_DD']=xdatawin
trackout['LAT_DD']=ydatawin
trackout['DEPTH']=zdatawin

# drop first rows (no values in moving average)
trackout=trackout.drop(trackout.index[range(windtime-1)])

# display your new table
trackout.head()

# output filename construction
fnout=(fn.replace(".csv", "_smoothed.csv"))
print(fnout)

# output
trackout.to_csv(fnout,index=False)

# prompt download
files.download(fnout)